### Search Engine with Tools and Agents

Reference: https://python.langchain.com/docs/integrations/tools/

### Tools
Tools are interfaces that an agent, chain or LLm can use to inteact with the world. They combine a few things:
- The name of the tool
- A description of what that tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the rsult of a tool should be returned directly to the user

#### Tool creation
- in-build tools: for wikipedia, arxiv and many more
- creating custm tools

In [5]:
# in-built tools
# wikipedia
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wikipedia=WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
print(wikipedia)

wikipedia.run("mukesh ambani")

api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\srish\\anaconda3\\envs\\langchain\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)


'Page: Mukesh Ambani\nSummary: Mukesh Dhirubhai Ambani (born 19 April 1957) is an Indian billionaire businessman who is the chairman and managing director of Reliance Industries. He is the richest perso'

In [ ]:
# in-built tools
# arxiv
from langchain.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=230)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.run("Generative AI")

'Published: 2021-03-29\nTitle: "Weak AI" is Likely to Never Become "Strong AI", So What is its Greatest Value for us?\nAuthors: Bin Liu\nSummary: AI has surpassed humans across a variety of tasks such as image\nclassification, playing '

In [9]:
# creating custom tools for RAG
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
hug_embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

loader=WebBaseLoader("https://docs.smith.langchain.com/")
web_loader=loader.load()
txt_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
chunks=txt_splitter.split_documents(web_loader)
faiss_vector=FAISS.from_documents(chunks,hug_embeddings)
retriever=faiss_vector.as_retriever()

In [10]:
# in order to use the retriever as a tool we need to convert it in a tool
# now to create tool we need to convert this retriever into tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"LangSmith-Search","Search anything about Langsmith")
retriever_tool.name

'LangSmith-Search'

In [11]:
tools=[wikipedia,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\srish\\anaconda3\\envs\\langchain\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=230)),
 Tool(name='LangSmith-Search', description='Search anything about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000205C07B7700>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020581B166

Reference: https://python.langchain.com/docs/concepts/agents/

### Agents
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded. In agents, a language model is used as a reasoning to determine which actions to take and in which order.

#### run all the tools with agents and LLM

to combine tools with LLM and then exceute them, we need AgentExecutor

In [12]:
# llm
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

groq_llm=ChatGroq(model="Llama3-8b-8192")

Reference :https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/

In [14]:
# new way of creating prompt template using hub
from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]